# Sprint11

# 今回の目的
Tensorflow のコードリーディングが目的<br>

In [1]:
# ミニバッチ処理
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]   # 特徴量の列
n_samples = X_train.shape[0] # 特徴量の行
n_classes = 1

# 計算グラフに渡す引数の形を決める
# shape [None, n_input]
# 変数宣言のみ
# 外部からの変数の中身を入れる
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        # Variable変数
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # tf.matmul 行列内積
    # tf.add 足し算
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # 活性化関数 relu
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits クロスエントロピー
# tf.reduce_mean 平均を算出
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# パラメータ最適化
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.equal 等号比較
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    # 初期化
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        # np.ceil 切り上げ
        # astype キャスト
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # パラメータ最適化
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc)) 
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 6.6157, val_loss : 11.0777, acc : 0.500, val_acc : 0.188
Epoch 1, loss : 9.1807, val_loss : 8.6315, acc : 0.500, val_acc : 0.625
Epoch 2, loss : 6.0724, val_loss : 5.6400, acc : 0.750, val_acc : 0.625
Epoch 3, loss : 3.7560, val_loss : 4.1699, acc : 0.750, val_acc : 0.438
Epoch 4, loss : 5.2472, val_loss : 4.9945, acc : 0.750, val_acc : 0.750
Epoch 5, loss : 6.1420, val_loss : 6.1444, acc : 0.750, val_acc : 0.750
Epoch 6, loss : 1.7371, val_loss : 1.5533, acc : 0.750, val_acc : 0.812
Epoch 7, loss : 0.0230, val_loss : 0.3402, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.7110, val_loss : 0.8361, acc : 0.750, val_acc : 0.812
Epoch 9, loss : 0.6809, val_loss : 0.8569, acc : 0.750, val_acc : 0.812
test_acc : 0.800


# 多値分類 （Iris）

## 多値分類用データ生成

In [3]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

## 正解ラベルをOne-Hot-Encoding

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
y_train_one_hot = enc.fit_transform(y_train)
y_test_one_hot = enc.transform(y_test)
y_val_one_hot = enc.fit_transform(y_val)

/home/hiro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/hiro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In

In [5]:
y_train_one_hot

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0

## 多値分類TF

In [6]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
# shape [None, n_input]
# 変数宣言のみ
# 外部からの変数の中身を入れる
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        # Variable変数
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # tf.matmul 行列内積
    # tf.add 足し算
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # 活性化関数 relu
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # 活性化関数 relu
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits クロスエントロピー
# tf.reduce_mean 平均を算出
softmax = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits)
loss_op = tf.reduce_mean(softmax)

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# パラメータ最適化
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.equal 等号比較
predict = tf.math.argmax(logits, axis=1)
#correct_pred = tf.equal(Y, logits)
correct_pred = tf.equal(tf.math.argmax(Y, axis=1), tf.math.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

#print("mini_batch_y:type".format(type(mini_batch_y)))
# 計算グラフの実行
with tf.Session() as sess:
    # 初期化
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        # np.ceil 切り上げ
        # astype キャスト
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # パラメータ最適化
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            acc = sess.run( accuracy, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc, val_pre = sess.run([loss_op, accuracy, predict], feed_dict={X: X_val, Y: y_val_one_hot})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}, val_pre : {}".format(epoch, loss, val_loss, acc, val_acc, val_pre)) 
    test_acc, val_pre = sess.run([accuracy, predict], feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}, val_pre : {}".format(test_acc, val_pre))

Epoch 0, loss : 5.1844, val_loss : 7.8531, acc : 0.667, val_acc : 0.458, val_pre : [2 1 0 0 0 0 0 0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
Epoch 1, loss : 30.6837, val_loss : 13.0076, acc : 0.333, val_acc : 0.708, val_pre : [2 1 2 1 1 1 1 1 2 1 1 1 2 1 2 1 2 2 1 2 2 1 1 1]
Epoch 2, loss : 0.0001, val_loss : 2.2452, acc : 1.000, val_acc : 0.792, val_pre : [2 1 2 0 0 0 1 1 2 0 0 0 2 0 2 1 0 2 0 2 2 0 1 1]
Epoch 3, loss : 3.0991, val_loss : 11.4867, acc : 0.667, val_acc : 0.625, val_pre : [2 0 2 0 0 0 0 0 2 0 0 0 2 0 2 0 0 2 0 2 2 0 0 0]
Epoch 4, loss : 0.0000, val_loss : 2.1597, acc : 1.000, val_acc : 0.792, val_pre : [2 1 2 0 0 0 1 1 2 0 0 0 2 0 2 1 0 2 0 2 2 0 1 1]
Epoch 5, loss : 0.0003, val_loss : 1.2098, acc : 1.000, val_acc : 0.875, val_pre : [2 1 2 0 0 0 1 1 2 0 0 1 2 1 2 1 0 2 0 2 2 0 1 1]
Epoch 6, loss : 1.7866, val_loss : 2.4999, acc : 0.667, val_acc : 0.875, val_pre : [2 1 2 0 0 0 1 1 2 1 1 1 2 1 2 1 0 2 1 2 2 1 1 1]
Epoch 7, loss : 1.9814, val_loss : 2.8836, acc : 0.667, val_acc : 

## MNIST
データセット生成

In [7]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

# 保存先を指定
mnist_dir = "./mnist_data/"

# MNISTの読み込み
mnist = fetch_mldata('MNIST original', data_home=mnist_dir)
# trainとtestに分割する
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.14285, shuffle=False)
# ラベルをint型にしておく
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

/home/hiro/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/home/hiro/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


## 前処理

In [8]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0

1.0
0.0


In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

(60000,)
(60000, 10)
float64


In [10]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
# shape [None, n_input]
# 変数宣言のみ
# 外部からの変数の中身を入れる
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        # Variable変数
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # tf.matmul 行列内積
    # tf.add 足し算
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # 活性化関数 relu
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits クロスエントロピー
# tf.reduce_mean 平均を算出
softmax = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits)
loss_op = tf.reduce_mean(softmax)

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# パラメータ最適化
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.equal 等号比較
predict = tf.math.argmax(logits, axis=1)
correct_pred = tf.equal(tf.math.argmax(Y, axis=1), tf.math.argmax(logits, axis=1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    # 初期化
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        # np.ceil 切り上げ
        # astype キャスト
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # パラメータ最適化
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            acc = sess.run( accuracy, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        #val_loss, val_acc, val_pre = sess.run([loss_op, accuracy, predict], feed_dict={X: X_val, Y: y_val_one_hot})
        #print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}, val_pre : {}".format(epoch, loss, val_loss, acc, val_acc, val_pre)) 
    test_acc, val_pre = sess.run([accuracy, predict], feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}, val_pre : {}".format(test_acc, val_pre))

test_acc : 0.921, val_pre : [0 0 0 ... 9 4 4]


## House Prices
データ読込み

In [11]:
# データセットの読み込み
dataset_path ="./HousePrices/train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)[:,np.newaxis]
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [12]:
X_train.shape

(934, 2)

In [13]:
y_train.shape

(934, 1)

In [14]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
# shape [None, n_input]
# 変数宣言のみ
# 外部からの変数の中身を入れる
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        # Variable変数
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # tf.matmul 行列内積
    # tf.add 足し算
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # 活性化関数 relu
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
recursion = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits クロスエントロピー
# tf.reduce_mean 平均を算出
loss_op = tf.reduce_mean(tf.square(Y - recursion))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# パラメータ最適化
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.equal 等号比較
predict = recursion
correct_pred = tf.square(Y - recursion)
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    # 初期化
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        # np.ceil 切り上げ
        # astype キャスト
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # パラメータ最適化
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            acc = sess.run( accuracy, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc)) 
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 1279344128.0000, val_loss : 2485468672.0000, acc : 1279344128.000, val_acc : 2485468672.000
Epoch 1, loss : 1314523520.0000, val_loss : 2537993984.0000, acc : 1314523520.000, val_acc : 2537993984.000
Epoch 2, loss : 1298465408.0000, val_loss : 2526713856.0000, acc : 1298465408.000, val_acc : 2526713856.000
Epoch 3, loss : 1250085120.0000, val_loss : 2513225216.0000, acc : 1250085120.000, val_acc : 2513225216.000
Epoch 4, loss : 1224210688.0000, val_loss : 2516456448.0000, acc : 1224210688.000, val_acc : 2516456448.000
Epoch 5, loss : 1201498624.0000, val_loss : 2506014208.0000, acc : 1201498624.000, val_acc : 2506014208.000
Epoch 6, loss : 1210170496.0000, val_loss : 2500556032.0000, acc : 1210170496.000, val_acc : 2500556032.000
Epoch 7, loss : 1201525632.0000, val_loss : 2496403712.0000, acc : 1201525632.000, val_acc : 2496403712.000
Epoch 8, loss : 1194140032.0000, val_loss : 2495319040.0000, acc : 1194140032.000, val_acc : 2495319040.000
Epoch 9, loss : 1204653696.0

In [15]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
# shape [None, n_input]
# 変数宣言のみ
# 外部からの変数の中身を入れる
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        # Variable変数
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # tf.matmul 行列内積
    # tf.add 足し算
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    # 活性化関数 relu
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
recursion = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits クロスエントロピー
# tf.reduce_mean 平均を算出
loss_op = tf.reduce_mean(tf.square(Y - recursion))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# パラメータ最適化
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.equal 等号比較
predict = recursion
correct_pred = tf.square(Y - recursion)

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    # 初期化
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        # np.ceil 切り上げ
        # astype キャスト
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # パラメータ最適化
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss)) 
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.3f}".format(test_loss))

Epoch 0, loss : 1316052608.0000, val_loss : 2512956928.0000
Epoch 1, loss : 1321143040.0000, val_loss : 2537303040.0000
Epoch 2, loss : 1317957376.0000, val_loss : 2553702912.0000
Epoch 3, loss : 1334023680.0000, val_loss : 2548765952.0000
Epoch 4, loss : 1289197696.0000, val_loss : 2552428288.0000
Epoch 5, loss : 1306729728.0000, val_loss : 2546171392.0000
Epoch 6, loss : 1250557824.0000, val_loss : 2540801280.0000
Epoch 7, loss : 1263934848.0000, val_loss : 2532036864.0000
Epoch 8, loss : 1220031616.0000, val_loss : 2525911296.0000
Epoch 9, loss : 1210066560.0000, val_loss : 2536782080.0000
test_loss : 3911321344.000


# Keras

In [16]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

Using TensorFlow backend.


In [17]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [18]:
enc = OneHotEncoder(sparse=False)
y_train_one_hot = enc.fit_transform(y_train)

/home/hiro/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
model = Sequential()
model.add(Dense(50, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 各イテレーションのバッチサイズを32で学習を行なう
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32)

Epoch 1/10
64/64 [==============================] - 1s 13ms/step - loss: nan - acc: 0.4688
Epoch 2/10
64/64 [==============================] - 0s 77us/step - loss: nan - acc: 0.4688
Epoch 3/10
64/64 [==============================] - 0s 81us/step - loss: nan - acc: 0.4688
Epoch 4/10
64/64 [==============================] - 0s 104us/step - loss: nan - acc: 0.4688
Epoch 5/10
64/64 [==============================] - 0s 135us/step - loss: nan - acc: 0.4688
Epoch 6/10
64/64 [==============================] - 0s 162us/step - loss: nan - acc: 0.4688
Epoch 7/10
64/64 [==============================] - 0s 74us/step - loss: nan - acc: 0.4688
Epoch 8/10
64/64 [==============================] - 0s 109us/step - loss: nan - acc: 0.4688
Epoch 9/10
64/64 [==============================] - 0s 117us/step - loss: nan - acc: 0.4688
Epoch 10/10
64/64 [==============================] - 0s 92us/step - loss: nan - acc: 0.4688


# Chainer Trainerなし

In [23]:
import numpy as np
import chainer
from chainer import backend
from chainer import backends
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [24]:
from __future__ import print_function
import matplotlib.pyplot as plt
from chainer.datasets import mnist

# Download the MNIST data if you haven't downloaded it yet
train, test = mnist.get_mnist(withlabel=True, ndim=1)

# Display an example from the MNIST dataset.
# `x` contains the input image array and `t` contains that target class
# label as an integer.
x, t = train[0]
plt.imshow(x.reshape(28, 28), cmap='gray')
plt.savefig('5.png')
print('label:', t)

label: 5


In [27]:
from chainer import iterators

# Choose the minibatch size.
batchsize = 128

train_iter = iterators.SerialIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize,
                                     repeat=False, shuffle=False)

In [29]:
class MyNetwork(Chain):

    def __init__(self, n_mid_units=100, n_out=10):
        super(MyNetwork, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def forward(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

model = MyNetwork()

gpu_id = -1  # Set to -1 if you use CPU
if gpu_id >= 0:
    model.to_gpu(gpu_id)

In [31]:
from chainer import optimizers

# Choose an optimizer algorithm
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)

# Give the optimizer a reference to the model so that it
# can locate the model's parameters.
optimizer.setup(model)

In [32]:
import numpy as np
from chainer.dataset import concat_examples
from chainer.backends.cuda import to_cpu

max_epoch = 10

while train_iter.epoch < max_epoch:

    # ---------- One iteration of the training loop ----------
    train_batch = train_iter.next()
    image_train, target_train = concat_examples(train_batch, gpu_id)

    # Calculate the prediction of the network
    prediction_train = model(image_train)

    # Calculate the loss with softmax_cross_entropy
    loss = F.softmax_cross_entropy(prediction_train, target_train)

    # Calculate the gradients in the network
    model.cleargrads()
    loss.backward()

    # Update all the trainable parameters
    optimizer.update()
    # --------------------- until here ---------------------

    # Check the validation accuracy of prediction after every epoch
    if train_iter.is_new_epoch:  # If this iteration is the final iteration of the current epoch

        # Display the training loss
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        test_losses = []
        test_accuracies = []
        while True:
            test_batch = test_iter.next()
            image_test, target_test = concat_examples(test_batch, gpu_id)

            # Forward the test data
            prediction_test = model(image_test)

            # Calculate the loss
            loss_test = F.softmax_cross_entropy(prediction_test, target_test)
            test_losses.append(to_cpu(loss_test.data))

            # Calculate the accuracy
            accuracy = F.accuracy(prediction_test, target_test)
            accuracy.to_cpu()
            test_accuracies.append(accuracy.data)

            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(test_losses), np.mean(test_accuracies)))

epoch:01 train_loss:0.2855 val_loss:0.2671 val_accuracy:0.9241
epoch:02 train_loss:0.2731 val_loss:0.1979 val_accuracy:0.9421
epoch:03 train_loss:0.1298 val_loss:0.1599 val_accuracy:0.9524
epoch:04 train_loss:0.1138 val_loss:0.1372 val_accuracy:0.9582
epoch:05 train_loss:0.1298 val_loss:0.1134 val_accuracy:0.9658
epoch:06 train_loss:0.1023 val_loss:0.1117 val_accuracy:0.9646
epoch:07 train_loss:0.0462 val_loss:0.1016 val_accuracy:0.9683
epoch:08 train_loss:0.1059 val_loss:0.0904 val_accuracy:0.9730
epoch:09 train_loss:0.1174 val_loss:0.0848 val_accuracy:0.9744
epoch:10 train_loss:0.0835 val_loss:0.0867 val_accuracy:0.9760


# Chainer Trainerあり

In [33]:
import numpy as np
import chainer
from chainer import backend
from chainer import backends
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [34]:
from chainer.datasets import mnist

train, test = mnist.get_mnist()

In [35]:
batchsize = 128

train_iter = iterators.SerialIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, False, False)

In [37]:
class MLP(Chain):

    def __init__(self, n_mid_units=100, n_out=10):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.Linear(None, n_mid_units)
            self.l3 = L.Linear(None, n_out)

    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

gpu_id = -1  # Set to -1 if you use CPU

model = MLP()
if gpu_id >= 0:
    model.to_gpu(gpu_id)

In [38]:
max_epoch = 10

# Wrap your model by Classifier and include the process of loss calculation within your model.
# Since we do not specify a loss function here, the default 'softmax_cross_entropy' is used.
model = L.Classifier(model)

# selection of your optimizing method
optimizer = optimizers.MomentumSGD()

# Give the optimizer a reference to the model
optimizer.setup(model)

# Get an updater that uses the Iterator and Optimizer
updater = training.updaters.StandardUpdater(train_iter, optimizer, device=gpu_id)

In [39]:
# Setup a Trainer
trainer = training.Trainer(updater, (max_epoch, 'epoch'), out='mnist_result')

In [41]:
from chainer.training import extensions

trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
trainer.extend(extensions.snapshot_object(model.predictor, filename='model_epoch-{.updater.epoch}'))
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.extend(extensions.dump_graph('main/loss'))

In [42]:
trainer.run()

epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           0.540782    0.848114       0.25758               0.925732                  3.11749       
1           0.540782    0.848114       0.25758               0.925732                  3.11749       
2           0.233239    0.931703       0.190733              0.942049                  7.29875       
2           0.233239    0.931703       0.190733              0.942049                  7.29875       
3           0.177396    0.948477       0.153954              0.954905                  11.8528       
3           0.177396    0.948477       0.153954              0.954905                  11.8528       
4           0.142438    0.9585         0.127018              0.963311                  16.4235       
4           0.142438    0.9585         0.127018              0.963311                 

# TensorFlow Tensorboard

In [48]:
writer = tf.summary.FileWriter("./tmp/tensorflow_log", sess.graph_def)
merged = tf.summary.merge_all()
#w_hist = tf.summary.histogram("weights", W)

# 以下はミニバッチ処理の実験

In [20]:
mini_batch_x.shape

(4, 2)

In [21]:
instance = GetMiniBatch(np.ones((150,10)), np.arange(150))

In [22]:
instance[4]

TypeError: 'GetMiniBatch' object does not support indexing

In [ ]:
for x, y in instance:
    pass

In [ ]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        #shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        #self.X = X[shuffle_index]
        #self.y = y[shuffle_index]
        self.X = X
        self.y = y
        #self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        print("len!")
        return self._stop
    def __iter__(self):
        print("iter!")
        self._counter = 0
        return self
    def __getitem__(self, item):
        print("getitem", item)
        print(type(item))
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __next__(self):
        print("next")
        if self._counter >= self._stop:
            #self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        print(p1, self.y[p0:p1])
        return self.X[p0:p1], self.y[p0:p1]

In [ ]:
instance.y.shape

In [ ]:
instance.y[900:1000]

In [ ]:
instance.y[1]

In [ ]:
np.arange(150)

In [ ]:
instance[1000:1050:3]